In [1]:
! pip install bertopic



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 2.1 MB/s 
     |████████████████████████████████| 5.2 MB 11.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 85 kB 3.8 MB/s 
     |████████████████████████████████| 88 kB 6.4 MB/s 
     |████████████████████████████████| 4.4 MB 42.5 MB/s 
     |████████████████████████████████| 1.2 MB 45.0 MB/s 
     |████████████████████████████████| 101 kB 10.6 MB/s 
     |████████████████████████████████| 636 kB 42.1 MB/s 
     |████████████████████████████████| 6.6 MB 36.4 MB/s 
     |████████████████████████████████| 1.1 MB 31.8 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2342160 sha256=03539738fb61a590d1ca66ea43ac32a5ec6e393ba0c8ef2344a420a1defb81c6
  Stored in directory: /root/.cache

In [2]:

from bertopic import BERTopic
import pandas as pd



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import re
import gensim
from nltk.stem import WordNetLemmatizer

punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'         # define a string of punctuation symbols

# Functions to clean tweets
def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
   # tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@)', '', tweet)  # remove tweeted at
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#)', ' ', tweet)  # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

def tokenize(tweet):
    """Returns tokenized representation of words in lemma form excluding stopwords"""
    result = []
    for token in gensim.utils.simple_preprocess(tweet):
        if token not in gensim.parsing.preprocessing.STOPWORDS \
                and len(token) > 2:  # drops words with less than 3 characters
            result.append(lemmatize(token))
    return result

def lemmatize(token):
    """Returns lemmatization of a token"""
    return WordNetLemmatizer().lemmatize(token, pos='v')

def preprocess_tweet(tweet):
    """Main master function to clean tweets, stripping noisy characters, and tokenizing use lemmatization"""
    #tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    #tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
  # tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet_token_list = tokenize(tweet)  # apply lemmatization and tokenization
    tweet = ' '.join(tweet_token_list)
    tweet = re.sub(' amp ', '&', tweet)
    return tweet

def basic_clean(tweet):
    """Main master function to clean tweets only without tokenization or removal of stopwords"""
    # tweet = remove_users(tweet)
    # tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet = re.sub(' & ', '', tweet)
    return tweet

def tokenize_tweets(df):
    df['tokens'] = df.tweet.apply(preprocess_tweet)
    num_tweets = len(df)
    print('Complete. Number of Tweets that have been cleaned and tokenized : {}'.format(num_tweets))
    return df

In [5]:
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/df_nlp_com5.csv",  lineterminator='\n')


### Topic modelling prime cinque comunità per numero di utenti

In [6]:
df1_com0 = df1[df1["community"]=="community_0"]
df1_com1 = df1[df1["community"]=="community_1"]
df1_com2 = df1[df1["community"]=="community_2"]
df1_com3 = df1[df1["community"]=="community_3"]
df1_com4 = df1[df1["community"]=="community_4"]


In [7]:
joinDf = [df1_com0, df1_com1, df1_com2, df1_com3, df1_com4]

In [8]:
df = pd.concat(joinDf)

In [9]:
df = df.reset_index(drop=True)

In [10]:
df

,id,conversation_id,created_at,date,time,timezone,user_id,name,place,tweet,...,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,community,original_community_extension
0,1513589455046774789,1513589455046774784,2022-04-11 19:47:01+01:00,2022-04-11,19:47:01,100,142015635,Larry G Putnam,NaN,"WOW, WOW, WOW #Biden Lied again! @angel_star39...",...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,community_0,74
1,1537580824626974720,1537353923279069184,2022-06-17 00:40:10+01:00,2022-06-17,00:40:10,100,142015635,Larry G Putnam,NaN,@Ultramagabella @rom08eo This is the the #Bide...,...,NaN,NaN,NaN,"[{'screen_name': 'Ultramagabella', 'name': 'Ul...",NaN,NaN,NaN,NaN,community_0,74
2,1522630146175512584,1522628336140357632,2022-05-06 18:31:30+01:00,2022-05-06,18:31:30,100,142015635,Larry G Putnam,NaN,"@angel_star39 I go Close in Selma,NC &amp; it ...",...,NaN,NaN,NaN,"[{'screen_name': 'angel_star39', 'name': 'Mona...",NaN,NaN,NaN,NaN,community_0,74
3,1519424310838022145,1519345034189058048,2022-04-27 22:12:39+01:00,2022-04-27,22:12:39,100,142015635,Larry G Putnam,NaN,"@marroelmonero @TamiP111 YES, A REPUBLIC: ""A s...",...,NaN,NaN,NaN,"[{'screen_name': 'marroelmonero', 'name': 'MAR...",NaN,NaN,NaN,NaN,community_0,74
4,1479785683178897409,1479785683178897408,2022-01-08 12:02:54+00:00,2022-01-08,12:02:54,100,142015635,Larry G Putnam,NaN,"Please Beam #biden Up, Scotty",...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,community_0,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,1504209386046201859,1003786405217296384,2022-03-16 21:33:59+00:00,2022-03-16,21:33:59,100,1344787579,GReno,NaN,@CanYouHearMeN15 @Fueldog3 @TheWreckingBal5 @D...,...,NaN,NaN,NaN,"[{'screen_name': 'CanYouHearMeN15', 'name': 'C...",NaN,NaN,NaN,NaN,community_3,48
360,1522327720348336129,1003786405217296384,2022-05-05 22:29:46+01:00,2022-05-05,22:29:46,100,1344787579,GReno,NaN,@account_walt @AnglerLouisiana @WCSmith5095257...,...,NaN,NaN,NaN,"[{'screen_name': 'account_walt', 'name': 'Walt...",NaN,NaN,NaN,NaN,community_3,48
361,1522309732945567746,1003786405217296384,2022-05-05 21:18:18+01:00,2022-05-05,21:18:18,100,1344787579,GReno,NaN,@WCSmith50952576 @FemmeVoilet @PogueMoran @Com...,...,NaN,NaN,NaN,"[{'screen_name': 'WCSmith50952576', 'name': 'W...",NaN,NaN,NaN,NaN,community_3,48
362,1504936821628747784,901233427525107712,2022-03-18 21:44:33+00:00,2022-03-18,21:44:33,100,884893716,Remember Otis (AKA: Killjoy),NaN,@Opsimath57 @VinceGottalotta @MJB24681 @flight...,...,NaN,NaN,NaN,"[{'screen_name': 'Opsimath57', 'name': 'Karen ...",NaN,NaN,NaN,NaN,community_4,48


In [11]:
df = df.drop_duplicates()

In [12]:
df["tweet"] = df.tweet.astype("str")

In [13]:
import nltk


In [14]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [15]:
df["preprocessed_tweet"] = df["tweet"].apply(preprocess_tweet)


In [16]:
df.value_counts()

Series([], dtype: int64)

In [17]:
df.community.value_counts()

community_1    241
community_0     82
community_2     33
community_3      6
community_4      2
Name: community, dtype: int64

In [18]:
tweets = df.preprocessed_tweet.to_list()
timestamp = df.date.to_list()

In [19]:
topic_model = BERTopic(language="english", nr_topics = 30)
topics, probs = topic_model.fit_transform(tweets)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [20]:
topic_model.get_topic_info()


,Topic,Count,Name
0,3,238,3_biden_potus_joebiden_trump
1,1,67,1_putin_russia_ukraine_biden
2,0,33,0_covid_approve_potus_fully
3,2,17,2_canyouhearmen_tom_neverwinter_trump
4,-1,9,-1_fmr_truck_usps_biden


In [21]:
topic_model.visualize_barchart()


In [22]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)


In [23]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)


In [24]:
 topic_model.get_representative_docs(0)


['potus potus political purge biden statement unvaccinated&fam suffer&die winter make fully approve early treatment available&half healthcare workers fire hospitals overwhelm want people die',
 'potus biden say end covid theres test state actively block state buy fully approve covid treatments&cause half healthcare workers fire point cdc say infect health workers dont quarantine',
 'potus trump vaccine year trump able thousands ventilators shift hospitals&vaccine experts say impossible biden test approve treatments available remember']

### Topic Modelling su tutti i tweet del quinto semestre

In [25]:
df1 = df1.drop_duplicates()

In [26]:
df1["tweet"] = df1.tweet.astype("str")

In [27]:
df1["preprocessed_tweet"] = df1["tweet"].apply(preprocess_tweet)


In [28]:
tweets = df1.preprocessed_tweet.to_list()
timestamp = df1.date.to_list()

In [29]:
topic_model = BERTopic(language="english", nr_topics = 30)
topics, probs = topic_model.fit_transform(tweets)

In [30]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,7849,-1_biden_joebiden_ukraine_potus
1,0,482,0_covid_vaccine_test_mask
2,1,342,1_inflation_economy_price_bidenflation
3,2,274,2_msnbc_cnn_foxnews_media
4,3,263,3_women_publishera_constitution_era
5,4,218,4_joebiden_joebide_man_like
6,5,187,5_border_illegals_illegal_migrants
7,6,170,6_gop_vote_election_voters
8,7,158,7_putin_dnc_russia_biden
9,8,155,8_china_taiwan_trade_tariff


In [31]:
topic_model.visualize_barchart()

In [32]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)

In [33]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

### Topic Modelling sulle prime cinque comunità per numero di tweets

In [34]:
df1['community'].value_counts()

community_451    489
community_294    379
community_103    242
community_1      241
community_5      238
                ... 
community_616      1
community_595      1
community_773      1
community_686      1
community_554      1
Name: community, Length: 880, dtype: int64

In [38]:
df1_com451 = df1[df1["community"]=="community_451"]
df1_com294 = df1[df1["community"]=="community_294"]
df1_com103 = df1[df1["community"]=="community_103"]
df1_com1 = df1[df1["community"]=="community_1"]
df1_com5 = df1[df1["community"]=="community_5"]



In [39]:
joinDf = [df1_com451, df1_com294, df1_com103, df1_com1, df1_com5]

In [40]:
df = pd.concat(joinDf)

In [41]:
df = df.reset_index(drop=True)

In [42]:
df = df.drop_duplicates()

In [43]:
df["tweet"] = df.tweet.astype("str")

In [44]:
df["preprocessed_tweet"] = df["tweet"].apply(preprocess_tweet)

In [45]:
df.community.value_counts()

community_451    489
community_294    379
community_103    242
community_1      241
community_5      238
Name: community, dtype: int64

In [46]:
tweets = df.preprocessed_tweet.to_list()
timestamp = df.date.to_list()



In [47]:
topic_model = BERTopic(language="english", nr_topics = 30)
topics, probs = topic_model.fit_transform(tweets)

In [48]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,452,-1_joebiden_biden_americans_america
1,0,253,0_women_publishera_constitution_era
2,2,230,2_ukraine_russia_putin_biden
3,11,130,11_usa_whitehouse_americans_kamalaharris
4,1,66,1_covid_approve_potus_fully
5,19,57,19_presssec_joebiden_whitehouse_jensucky
6,13,54,13_biden_fck_brooklyn_administration
7,7,48,7_iran_israel_biden_deal
8,16,41,16_bidengashike_gas_joebiden_gasprices
9,18,38,18_joebiden_easterbunny_like_wonder


In [49]:
topic_model.visualize_barchart()

In [50]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)

In [51]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)